In [1]:
# preamble
from datetime import datetime
start = datetime.now()

## 4.5 Data Consistency Checks

## Step 01 > Consistency checks from exercise 4.5

### Create a dataframe

In [2]:
#import libraries
import pandas as pd
import numpy as np
import os 

df_test = pd.DataFrame()

### Create a mixed type column

In [3]:
df_test['mix'] = ['a','b',1,True]
for col in df_test.columns.tolist():
  weird = (df_test[[col]].applymap(type) != df_test[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_test[weird]) > 0:
    print (col)

mix


### Import ```products.csv```

In [4]:
#path pointing to the root directory fow Jupyter Notebooks in this PC
IC_path = r'C:\Users\joseh\Instacart Basket Analysis'
df_prods = pd.read_csv(os.path.join(IC_path,'02 Data','Original Data','products.csv'))

In [5]:
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

### identifying the entries with NULL product_name 

In [6]:
#filter (slice) only those producs for which product_name is null
df_prods_null_name = df_prods[df_prods['product_name'].isnull() == True]

In [7]:
# verifying the shape of df_prods
df_prods.shape
# (49693, 5)

(49693, 5)

In [8]:
df_prods_clean = df_prods[df_prods['product_name'].isnull()==False]

In [9]:
df_prods_clean.shape
## (49677, 5) # 16 rows less 

(49677, 5)

In [10]:
#finding duplicates
df_dups = df_prods_clean[df_prods_clean.duplicated()]

df_dups #displaying the duplicates

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


In [11]:
#removing duplicates
df_prods_clean_no_dups = df_prods_clean.drop_duplicates()

In [12]:
df_prods_clean_no_dups.shape
# 5 rows eliminated
# the cleaned data set will be exported at the very end of this notebook

(49672, 5)

## Step 02 > Run the ```df.describe()``` function on your ```df_ords``` dataframe. Using your new knowledge about how to interpret the output of this function, share in a markdown cell whether anything about the data looks off or should be investigated further.
Tip: Keep an eye on min and max values!


In [13]:
#loading df_ords
df_ords = pd.read_csv(os.path.join(IC_path,'02 Data','Original Data','orders.csv'))
df_ords.describe()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


#### Findings
1. All columns, except for ```days_since_prior_order```, there are 3,421,803 records.
2. In the column ```days_since_prior_order``` there are 206,209 records (6% of all records) with "NULL". This does not look like an anomally because the "NULL" is most likely for the first time a customer places an order
3. The number of NULLs stated in item 1 are consistent with the number of user_ids being 206,209 (max ```user_id```)
4. ```order_dow``` ranges from 0 to 6. I have made the assumption that 0=Sunday and 6=Saturday; however, this needs to be confirmed

Based on the output of ```df.describe()``` I did not find any evidence to investigate further

In [14]:
#eliminate the column "eval_set"
df_ords = df_ords.drop(columns=['eval_set'])
df_ords

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0
...,...,...,...,...,...,...
3421078,2266710,206209,10,5,18,29.0
3421079,1854736,206209,11,4,10,30.0
3421080,626363,206209,12,1,12,18.0
3421081,2977660,206209,13,1,12,7.0


In [15]:
#finding 2 from the list above
df_ords['days_since_prior_order'].isnull().sum()

206209

## Step 03 > Check for mixed-type data in your ```df_ords``` dataframe.

In [16]:
#running code to determine which columns have mixed type
#this is done by comparing the data type of the first entry to all remaining entries
#and flagging any column for which there's at least one entry with a different data type
for col in df_ords.columns.tolist():
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords[weird]) > 0:
    print (col)

In [17]:
df_ords.dtypes

order_id                    int64
user_id                     int64
order_number                int64
order_dow                   int64
order_hour_of_day           int64
days_since_prior_order    float64
dtype: object

__I did not find mixed-type data__

## Step 04 > If you find mixed-type data, fix it. The column in question should contain observations of a single data type

__I did not find mixed-type data__

## Step 05 > Run a check for missing values in your ```df_ords``` dataframe.

In [18]:
df_ords.shape 

(3421083, 6)

In [19]:
df_ords.describe()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


In [20]:
#counting nulls for every column (if any)
df_ords.isnull().sum()

order_id                       0
user_id                        0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

* __The only column with missing values is ```days_since_prior_order```__
* These records represent the first order placed for a given ```user_id```. For such cases there is no reference of a previous order so we can determine the number of days since prior order

In [21]:
# create a new column named "first_order_from_customer"
# the value it holds is boolean indicating if the records in question is a firt order (True) or not
df_ords['first_order_from_customer'] = df_ords['days_since_prior_order'].isnull()
df_ords

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,first_order_from_customer
0,2539329,1,1,2,8,NaN,True
1,2398795,1,2,3,7,15.0,False
2,473747,1,3,3,12,21.0,False
3,2254736,1,4,4,7,29.0,False
4,431534,1,5,4,15,28.0,False
...,...,...,...,...,...,...,...
3421078,2266710,206209,10,5,18,29.0,False
3421079,1854736,206209,11,4,10,30.0,False
3421080,626363,206209,12,1,12,18.0,False
3421081,2977660,206209,13,1,12,7.0,False


## Step 06 > Address the missing values using an appropriate method.

* In the cell above I created a separate column with boolean values (True/False) to indicate whether the record in question represents the first order for a given customer

## Step 07 > Run a check for duplicate values in your ```df_ords``` data.
* I did not find any duplicate rows in the df_ords data set

In [22]:
#determining if there are any duplicate rows in df_prds
df_ords[df_ords.duplicated()]

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,first_order_from_customer


## Step 08 > Address the duplicates using an appropriate method.

* Nothing to do since there are no duplicates

## Step 09 > Export your final, cleaned ```df_prods``` and ```df_ords``` data as “.csv” files in your “Prepared Data” folder and give them appropriate, succinct names.

In [23]:
#exporting to csv in the folder _Prepared_Data_
df_prods_clean_no_dups.to_csv(os.path.join(IC_path,'02 Data','Prepared Data','products_checked.csv'))
df_ords.to_csv(os.path.join(IC_path,'02 Data','Prepared Data','orders_checked.csv'))

In [24]:
# fyi only - to track performance of this notebook 
print(' time to run this notebook h:mm:ss: ', datetime.now() - start)

 time to run this notebook h:mm:ss:  0:00:19.844520


In [26]:
df_ords.shape

(3421083, 7)

In [27]:
df_prods_clean_no_dups.shape

(49672, 5)

In [28]:
df_prods.shape

(49693, 5)